In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

url = "https://www.jitta.com/stock/bkk:com7/factsheet"

cookies = {'JDCID':'e9c939968e2294fbc49357a8dc17ad64898182c0ca16abc003b420bf1e851e00.5439c914472955e6ecc4ac72cdffa7c689a9020dc3d869f6b0b63278cd6d543e715726157f5bbd49210a4b2cc4dec3851703061b2613e31945142b61f94210a169b437eb4054a10db920a0f8f433b0620c295848890c'}

result = requests.get(url, cookies = cookies)
soup = BeautifulSoup(result.text, features='html.parser')

# with open('page.html','w', encoding='utf-8') as f:
#     f.write(result.text)


data = {}
row_years = list(map(lambda x: x.getText() , soup.find_all('div', {'class':'Text__TextSM-dn2wcp-2 FactsheetTableHeader__FlexItem-efsh34-3 fbxBgM'})))
row_names = list(map(lambda x: x.getText() , soup.find_all('div', {'style': 'white-space:nowrap;overflow:hidden;text-overflow:ellipsis;padding:8px'})))
row_datas = list(map(lambda x: x.getText() , soup.find_all('span',{'class': 'FactsheetTableRow__TooltipWrapper-sc-1wziwtk-6 bMjxnb'})))
chunk_size = len(row_years)

for idx,row_name in enumerate(row_names):
    data[row_name] = []
    for i in range(idx*chunk_size, (idx+1)*(chunk_size)):
        data[row_name].append(row_datas[i])

pd_data = pd.DataFrame(data)
pd_data.index = row_years

In [2]:
for column in pd_data.columns:
    pd_data[column] = pd_data[column].str.replace(',','').str.replace('%','')
    pd_data[column] =  pd.to_numeric(pd_data[column], errors = 'coerce')

In [3]:
# calculate more data
pd_data.loc[:, 'อัตราค่าใช้จ่ายในการขายและบริหาร'] = pd_data.loc[:, 'ค่าใช้จ่ายในการขายและบริหาร'] * 100 / pd_data.loc[:, 'รายได้รวม']

In [4]:
pd_data

,ราคา,อัตราส่วนราคาต่อกำไร,รายได้ต่อหุ้น,กำไรต่อหุ้น,เงินปันผลต่อหุ้น,ค่าใช้จ่ายในการลงทุนต่อหุ้น,มูลค่าตามบัญชีต่อหุ้น,จำนวนหุ้นสามัญที่เรียกชำระแล้ว,รายได้รวม,กำไรขั้นต้น,...,อัตราผลตอบแทนจากสินทรัพย์,อัตราส่วนสภาพคล่อง,หนี้สินต่อส่วนของผู้ถือหุ้น,อัตราการจ่ายเงินปันผล,วงจรเงินสด,เงินสดจากกิจกรรมดำเนินงาน,เงินสดจากกิจกรรมลงทุน,กระแสเงินสดที่ได้จากการจัดหาเงิน,เงินสดเปลี่ยนแปลงสุทธิ,อัตราค่าใช้จ่ายในการขายและบริหาร
2014,NaN,NaN,5.87,0.087,0.000,0.061,0.22,2400.0,14099.0,1700.0,...,5.8,0.88,7.40,0.0,17.2,-455.500,-359.2,736.3,-78.4,9.915597
2015,2.58,24.3,6.24,0.130,0.075,0.055,0.74,2400.0,14986.0,1911.0,...,6.1,1.34,1.49,56.9,31.9,764.300,-124.7,-164.6,475.0,10.236220
2016,6.55,34.8,7.16,0.170,0.130,0.092,0.83,2400.0,17177.0,2388.0,...,8.8,1.26,1.43,73.8,30.7,311.200,-399.9,69.2,-19.4,10.875007
2017,8.15,34.3,9.40,0.250,0.180,0.080,0.96,2400.0,22558.0,3050.0,...,10.1,1.17,2.12,69.0,17.6,-0.091,-366.8,353.5,-13.4,10.080681
2018,7.75,20.5,11.60,0.370,0.250,0.059,1.15,2400.0,27953.0,3666.0,...,12.1,1.25,1.74,67.6,33.4,1001.000,-240.3,-398.8,361.5,9.086681
2019,13.30,26.7,13.90,0.510,0.300,0.070,1.41,2400.0,33390.0,4444.0,...,14.5,1.28,1.71,59.4,29.6,726.200,-338.8,-142.4,245.1,8.760108
2020,19.50,34.9,15.60,0.620,0.500,0.089,1.61,2400.0,37353.0,4757.0,...,16.2,1.32,1.39,80.6,38.5,2308.000,-515.9,-2224.0,-431.9,7.994003
2021,40.90,35.7,21.30,1.100,0.500,0.000,2.20,2400.0,51151.0,6870.0,...,20.9,1.22,2.02,45.5,24.1,704.800,-727.7,817.7,794.8,7.225665
2022,34.00,26.8,26.40,1.270,0.750,0.000,3.00,2385.0,62981.0,8683.0,...,17.3,1.20,1.66,59.1,42.6,1644.000,-2435.0,463.4,-327.5,7.295851
2023,26.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
interesting_columns =  [
                    'รายได้รวม',
                    'กำไรต่อหุ้น',
                    'อัตรากำไรขั้นต้น',
                     'อัตราค่าใช้จ่ายในการขายและบริหาร',
                     'หนี้สินระยะยาว',
                     'หนี้สินต่อส่วนของผู้ถือหุ้น',
                     'อัตราผลตอบแทนผู้ถือหุ้น',
                     'อัตราการจ่ายเงินปันผล',
                    'อัตราส่วนราคาต่อกำไร',
                    'ราคา',
                    'เงินปันผลต่อหุ้น'
]
pd_data.index.name = 'ปี'
pd_data = pd_data[interesting_columns]

In [17]:
import string
from openpyxl.styles import Color, PatternFill, Font, Border

column_offset = 10
row_offset = 17
start_column = 1

column_mapping = list(string.ascii_uppercase) + list(map(lambda x: 'A'+x, string.ascii_uppercase))
column_position = start_column
row_position = 20
limit_col = 26


excel_file = 'output.xlsx'
sheet_name = 'COM7'

bar_chart = set([
        'รายได้รวม',
        'กำไรต่อหุ้น',
        'หนี้สินระยะยาว',
        'อัตราการจ่ายเงินปันผล'
])

line_chart = set([
        'อัตรากำไรขั้นต้น',
        'อัตราค่าใช้จ่ายในการขายและบริหาร',
        'หนี้สินต่อส่วนของผู้ถือหุ้น',
        'อัตราผลตอบแทนผู้ถือหุ้น'
])

no_plot_chart = set([
    'ราคา',
     'อัตราส่วนราคาต่อกำไร',
     'เงินปันผลต่อหุ้น'
])



with pd.ExcelWriter('output.xlsx',  engine="xlsxwriter") as writer:  
    pd_data.to_excel(writer, sheet_name=sheet_name)
    
    # plot graph
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    
    # financial data
    # fill red color for separate
    row_separate = 16
    format = workbook.add_format()
    format.set_pattern(1)
    format.set_bg_color('red')
    for column in column_mapping:
        worksheet.write(f'{column}{row_separate}', '',format)
    worksheet.write(f'A{row_separate+1}', 'Financial Data')
    
    start_row = 1
    end_row = len(pd_data)
    for column in pd_data.columns:
        col_x = 0
        col_y = pd_data.columns.get_loc(column) + 1
        chart = None
        if column in bar_chart:
            chart = workbook.add_chart({'type': 'column'})
        elif column in line_chart:
            chart = workbook.add_chart({'type': 'line'})
        elif column in no_plot_chart:
            continue
        else:
            raise Exception('cannot decide which graph should show')
            
        chart.add_series({
            'categories': [sheet_name, start_row, col_x, end_row, col_x], # ปี
            'values':     [sheet_name, start_row, col_y, end_row, col_y], #data
        })
        # Configure the chart axes.
        chart.set_x_axis({'name':  'ปี', 'position_axis': 'on_tick'})
        chart.set_y_axis({'name': column, 'major_gridlines': {'visible': True}})
        # Turn off chart legend. It is on by default in Excel.
        chart.set_legend({'position': 'off'})
        chart.set_title({'name': f'{column}'})

        # Insert the chart into the worksheet.
        worksheet.insert_chart(f'{column_mapping[column_position]}{row_position}', chart)
        
        # shift for next graph
        column_position += column_offset
        if column_position >= limit_col:
            column_position = start_column
            row_position += row_offset
    
    
    # P/E stock evaluation
    # fill red color for separate
    row_separate = 73
    format = workbook.add_format()
    format.set_pattern(1)
    format.set_bg_color('red')
    for column in column_mapping:
        worksheet.write(f'{column}{row_separate}', '',format)
    worksheet.write(f'A{row_separate+1}', 'P/E evaluation')
    
    current_year = '2023'
    last_year = str(int(current_year) - 1)
    pe_data = pd_data.loc[:,['อัตราส่วนราคาต่อกำไร','กำไรต่อหุ้น','ราคา']].T
    expected_earning = pe_data.loc['กำไรต่อหุ้น',last_year]
    pe_data.to_excel(writer, sheet_name=sheet_name, startrow=row_separate + 3, startcol=1)
    
    pe = list(pe_data.loc['อัตราส่วนราคาต่อกำไร',:].dropna())
    assert(len(pe) > 5)
    pe_sum = sum(pe) - max(pe) - min(pe)
    avg_pe = pe_sum / len(pe)-2
    fair_price = expected_earning * avg_pe
    mos30_price = 0.7 * fair_price
    current_price = pe_data.loc['ราคา', current_year]
    
    current_row = 83
    data_write = {
        'AVG P/E': avg_pe,
        'Fair Price': fair_price,
        'MOS 30 Price': mos30_price,
        'Current price': current_price
    }
    for k,v in data_write.items():
        worksheet.write(f'B{current_row}', k)
        worksheet.write(f'C{current_row}', v)
        current_row += 1
        
    
    # DDM stock evaluation
    # fill red color for separate
    row_separate = 88
    format = workbook.add_format()
    format.set_pattern(1)
    format.set_bg_color('red')
    for column in column_mapping:
        worksheet.write(f'{column}{row_separate}', '',format)
    worksheet.write(f'A{row_separate+1}', 'DDM evaluation')
    
    ddm_data = pd_data.loc[:,['เงินปันผลต่อหุ้น','ราคา']].T
    ddm_data.to_excel(writer, sheet_name=sheet_name, startrow=row_separate + 3, startcol=1)
    
    dividend = ddm_data.loc['เงินปันผลต่อหุ้น', last_year]
    expected_return = 0.1
    growth_rate = 0.03
    price_to_buy = (dividend * (1+growth_rate)) / (expected_return - growth_rate)
    current_price = ddm_data.loc['ราคา', current_year]
    
    data_write = {
        'Dividend': dividend,
        'Expected Return': expected_return,
        'Growth Rate': growth_rate,
        'Price to Buy': price_to_buy,
        'Current Price': current_price
    }
    current_row = 97
    for k,v in data_write.items():
        worksheet.write(f'B{current_row}', k)
        worksheet.write(f'C{current_row}', v)
        current_row += 1
    
    
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()    
writer.close()

C:\Users\ekala\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [18]:
import datetime
year = datetime.date.today().year
    

In [19]:
year

2023